In [1]:
"""
.. _example-usage:

Simple Step-By-Step Example on How to do ML with torchpme
=========================================================

.. currentmodule:: torchpme

This example showcases how the main capabilities of ``torchpme``

"""


'\n.. _example-usage:\n\nSimple Step-By-Step Example on How to do ML with torchpme\n=========================================================\n\n.. currentmodule:: torchpme\n\nThis example showcases how the main capabilities of ``torchpme``\n\n'

In [2]:
import numpy as np
from ase import Atoms
import chemiscope
import torch
from vesin.torch import NeighborList

from torchpme import EwaldCalculator
from torchpme.potentials import CoulombPotential


# 

# Generate Simple Example Structures
Throughout this tutorial, we will work with a simple atomic structure in three dimensions, which is a distorted version of the CsCl structure.

In [3]:
device = 'cpu' # alternatively: 'gpu'
dtype = torch.float64 # our neighbor list calculator requires torch.float64

In [4]:
# Generate a single unit cell of CsCl (2 atoms in the cell)
structure_single = Atoms('CsCl', cell=np.eye(3), positions=[[0,0,0],[0.5,0.5,0.5]])

# Generate a big structure by periodically repeating the unit cell and apply a random
# distortion to all atoms
structure_big = structure_single.repeat(3)
structure_big.rattle(stdev=0.01)
structure_big.wrap() # make sure all atoms are in the unit cell

# Get the required variables for later use and define the charges for each atom:
# all Cs atoms get a charge of +1, all Cl atoms a charge of -1
positions = torch.tensor(structure_big.positions, dtype=dtype, device=device, requires_grad=True)
cell = torch.tensor(structure_big.cell, dtype=dtype, device=device)
num_atoms = len(positions)
charges = torch.ones((num_atoms,1), dtype=dtype, device=device)
charges[::2] *= -1

/tmp/ipykernel_1938614/4073243837.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  cell = torch.tensor(structure_big.cell, dtype=dtype, device=device)


In [5]:
# Visualize structure
# chemiscope.show(structure_big, meta=dict(name="CsCl structure"))

# Tuning: Find Optimal Hyperparameters
Ewald and mesh methods require the specification of multiple hyperparameters, namely
1. the cutoff radius $r_\mathrm{cut}$ for the short-range parts
2. the smearing parameter $\sigma$ determining the relative importance of the short-range and long-range terms in the split
3. either the mesh spacing $h$ for mesh-based methods, or a reciprocal space cutoff $k_\mathrm{cut} = 2\pi/\lambda$ for the Ewald sum, where $\lambda$ is the shortest wavelength used in the Fourier series and corresponds to $h$ for mesh-based approaches

For ML applications, we typically first select a short-range cutoff similarly to conventional short-ranged ML models, and define the remaining parameters from there. In this example, we are simply computing the Coulomb potential, and thus compute the hyperparameters simply based on convergence criteria.

In [6]:
box_length = cell[0,0]
rcut = box_length.clone().detach() / 2 - 1e-10
smearing = rcut / 5
lr_wavelength = smearing / 2 # lambda which gives the reciprocal space cutoff kcut=2*pi/lambda

# Define Potential
We now need to define the potential function with which the atoms interact. Since this is a library for long-range ML, we support three major options:
1. the Coulomb potential ($1/r$)
2. more general inverse power-law potentials ($1/r^p$)
3. an option to build custom potentials using splines

This tutorial focuses on option (1), which is the most relevant from a practical point of view. We can simply initialize an instance of the `CoulombPotential` class that contains all the necessary functions (such as those defining the short-range and long-range splits) for this potential and makes them useable in the rest of the code.

In [7]:
potential = CoulombPotential(smearing = smearing)

/home/khugueni/code/torch-pme/src/torchpme/potentials/potential.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "smearing", torch.tensor(smearing, device=device, dtype=dtype)


# 

# Neighbor List
`torchpme` requires us to compute the neighbor list (NL) in advance. This is because for many ML applications, we would otherwise need to repeat this computation multiple times during model training of a neural network etc. By computing it externally and providing it as an input, we can streamline training workflows.

Note that we are not directly returning the distances themselves, but rather the 'neighbor shifts' as is indicated by the quantity 'S'. This is because we are typically still interested in taking gradients with respect to the atomic positions. An exact definition can be found in the `vesin` documentation.


In [ ]:
nl = NeighborList(cutoff=rcut, full_list=False)
i, j, neighbor_distances = nl.compute(points=positions, box=cell, periodic=True, quantities="ijd")
neighbor_indices = torch.stack([i, j], dim=1)

# Main Part: Calculator
The `Calculator` classes are the main user-facing classes in `torchpme`. These are used to compute atomic potentials $V_i$ for a given set of positions and particle weights (charges). For periodic calculators that are the main focus of this tutorial, it is also required to specify a `cell`.

In [9]:
# Initialization
calculator = EwaldCalculator(potential=potential, lr_wavelength=lr_wavelength)

# Compute Energy
We have now all ingredients: we can use the `Calculator` class to, well, actually compute the potentials $V_i$ at the position of the atoms, or the total energy for the given particle weights (charges). The electrostatic potential can then be obtained as
\begin{align}
E = \sum_{i=1}^N q_i V_i
\end{align}

In [10]:
potentials = calculator.forward(charges, cell, positions, neighbor_indices, neighbor_distances)
energy = torch.sum(charges * potentials)

# Compute Forces using backpropagation (automatic differentiation)

The forces on the particles can simply be obtained as minus the gradient of the energy with respect to the positions. These are easy to evaluate using the automatic differentiation capabilities of `pytorch` using the backpropagation method.

In [11]:
energy.backward()
forces = -positions.grad # forces on the particles

# Aperiodic Structures
For now, we have been using the `EwaldCalculator` which is a periodic calculator. We can however also use it for aperiodic structures by just using it as a calculator with a cutoff radius.

In [12]:
# Clone the positions to avoid accumulating gradients with respect to the same
# variables multiple times
positions_aperiodic = positions.clone().detach()
positions_aperiodic.requires_grad = True

# Compute neighbor list but this time without periodic boudary conditions
i, j, neighbor_distances_aperiodic = nl.compute(points=positions_aperiodic, box=cell, periodic=True, quantities="ijd")
neighbor_indices_aperiodic = torch.stack([i, j], dim=1)

# Compute aperiodic potential
potentials_aperiodic = calculator._compute_rspace(charges,
                                                  neighbor_indices_aperiodic,
                                                  neighbor_distances_aperiodic)

# Compute total energy and forces
energy_aperiodic = torch.sum(charges * potentials_aperiodic)
energy_aperiodic.backward()
forces_aperiodic = positions_aperiodic.grad # forces on the particles

# References to more advanced tasks